In [29]:
import pandas as pd
from functools import reduce

df_order_report = pd.read_csv('returns_\ca_export.csv')
# df_item_export = pd.read_excel('returns_\InventoryExport_3-16-2022-14-57-22-475.xlsx')

#original format
# df_returns_export = pd.read_csv('returns_\\report-1714490019067.tsv', sep="\t",encoding= 'unicode_escape')
df_returns_export = pd.read_csv('returns_\\amz_export.csv',encoding= 'unicode_escape')

df_order_report = df_order_report.rename(columns={"Site Order ID": "Order ID"})



In [30]:
inner_join = pd.merge(
    df_returns_export,
    df_order_report,
    on='Order ID',
    how='inner'
)

inner_join
# df_returns_export.groupby('Tracking ID')['tempx'].apply(' '.join).reset_index()

,Order ID,Order date,Return request date,Return request status,Amazon RMA ID,Merchant RMA ID,Label type,Label cost,Currency code,Return carrier,...,SKU,Title,Order Date,Shipment Status Date,Payment Type,Quantity,Order Total,Shipping Status,Fulfillment,ChannelAdvisor Order ID
0,113-5592034-4389824,09-Mar-2022,17-Mar-2022,Approved,DYypNPxGRRMA,,AmazonPrePaidLabel,6.22,USD,UPS,...,D97979-SZ-6.5,"Adidas Nemeziz 18.2 Fg Mens Shoes Size 6.5, Co...",3/9/2022 2:17 PM,3/10/2022 8:28 AM,Amazon,1,$160.83,Shipped,Seller Managed,19067898
1,113-5592034-4389824,09-Mar-2022,17-Mar-2022,Approved,DYypNPxGRRMA,,AmazonPrePaidLabel,6.22,USD,UPS,...,DA9335-SZ-6.5,"Adidas X 18.3 FG Mens Shoes Size 6.5, Color: F...",3/9/2022 2:17 PM,3/10/2022 8:28 AM,Amazon,1,$160.83,Shipped,Seller Managed,19067898
2,113-5592034-4389824,09-Mar-2022,17-Mar-2022,Approved,DYypNPxGRRMA,,AmazonPrePaidLabel,6.22,USD,UPS,...,D97979-SZ-6.5,"Adidas Nemeziz 18.2 Fg Mens Shoes Size 6.5, Co...",3/9/2022 2:17 PM,3/10/2022 8:28 AM,Amazon,1,$160.83,Shipped,Seller Managed,19067898
3,113-5592034-4389824,09-Mar-2022,17-Mar-2022,Approved,DYypNPxGRRMA,,AmazonPrePaidLabel,6.22,USD,UPS,...,DA9335-SZ-6.5,"Adidas X 18.3 FG Mens Shoes Size 6.5, Color: F...",3/9/2022 2:17 PM,3/10/2022 8:28 AM,Amazon,1,$160.83,Shipped,Seller Managed,19067898
4,114-1101030-8377053,15-Feb-2022,17-Mar-2022,Approved,DfyBN1xnRRMA,,AmazonPrePaidLabel,8.46,USD,UPS,...,EG8193-SZ-6.5,"Adidas X Ghosted.3 Fg J Mens Shoes Size 6.5, C...",2/15/2022 1:22 PM,2/16/2022 7:46 AM,Amazon,1,$74.35,Shipped,Seller Managed,19031473
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
791,113-1512943-1750637,02-Mar-2022,10-Mar-2022,Approved,DTGrR2xjRRMA,,AmazonPrePaidLabel,5.90,USD,UPS,...,D97940-SZ-7.5,"Adidas Predator 19.2 Fg Mens Shoes Size 7.5, C...",3/2/2022 12:03 PM,3/3/2022 8:21 AM,Amazon,1,$94.66,Shipped,Seller Managed,19055801
792,113-3967540-1926615,02-Mar-2022,10-Mar-2022,Approved,DkG1RGx8RRMA,,AmazonPrePaidLabel,5.52,USD,UPS,...,FV9060-SZ-12,"Adidas Speed Trainer 5 Mens Shoes Size 12, Col...",3/2/2022 8:14 AM,3/2/2022 1:11 PM,Amazon,1,$74.34,Shipped,Seller Managed,19055497
793,114-4977793-5485013,06-Mar-2022,10-Mar-2022,Approved,D2GtRqxTRRMA,,AmazonPrePaidLabel,8.46,USD,UPS,...,EG5633-SZ-11,"Adidas Afterburner 7 Mens Shoes Size 11, Color...",3/6/2022 5:59 AM,3/7/2022 8:54 AM,Amazon,1,$82.76,Shipped,Seller Managed,19061396
794,113-7740404-7102626,02-Feb-2022,10-Mar-2022,Approved,DKGGRGxnRRMA,,AmazonPrePaidLabel,5.52,USD,UPS,...,MSS-1110519BHBI-SZ-11,"HOKA ONE ONE Bondi 7 Womens Shoes Size 11, Col...",2/2/2022 5:20 PM,2/3/2022 8:31 AM,Amazon,1,$201.29,Shipped,Seller Managed,19014410
